Predictive Maintenance : Training Hyperparameter and Prediction

In [2]:
%store -r
%store

Stored variables and their in-db values:
create_date                         -> '22-02-01-15-40-34'
data_channel                        -> {'train': <sagemaker.inputs.TrainingInput object a
dw_output_path_prm                  -> 's3://ml-predictivemaintainaince-28-01-2022/export
exp_prefix                          -> 'sagemaker-experiments/linear-learner-22-02-01-15-
features_created_prm                -> True
path_to_test_data_prm               -> 's3://ml-predictivemaintainaince-28-01-2022/test/t
path_to_test_x_data_prm             -> 's3://ml-predictivemaintainaince-28-01-2022/test/t
path_to_train_data_prm              -> 's3://ml-predictivemaintainaince-28-01-2022/train/
path_to_valid_data_prm              -> 's3://ml-predictivemaintainaince-28-01-2022/valida


In [5]:
!pip install -qU 'sagemaker-experiments' 'sagemaker' 'boto3' 'awswrangler'

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.21.5 which is incompatible.
awscli 1.22.23 requires botocore==1.23.23, but you have botocore 1.23.49 which is incompatible.
awscli 1.22.23 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.22.23 requires rsa<4.8,>=3.1.2, but you have rsa 4.8 which is incompatible.
aiobotocore 2.0.1 requires botocore<1.22.9,>=1.22.8, but

In [36]:
import os
import json
import sys
import collections
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#sagemaker dependencies

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve
#import awswrangler as wr

In [37]:
#instantiate a sagemaker session
smclient = boto3.Session().client("sagemaker")
region = boto3.Session().region_name

#IAM role that is assigned
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = "ml-predictivemaintainaince-28-01-2022"

prefix_prm = "predmaint"

Train

In [6]:
from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker
from time import strftime

In [10]:
if 'create_date' not in locals():
    create_date = strftime("%y-%m-%d-%H-%M-%S")
    %store create_date
    
        # location within S3 for outputs
    exp_prefix = f'sagemaker-experiments/linear-learner-{create_date}'
    %store exp_prefix

In [13]:
b = locals()
print(b)

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'get_ipython().run_cell_magic(\'sh\', \'\', \' which sm_analytics_runtime_check >/dev/null 2>&1 && sm_analytics_runtime_check 2>/dev/null|| echo "{}"\\n\')', "get_ipython().run_line_magic('store', '-r')\nget_ipython().run_line_magic('store', '')", 'import os\nimport json\nimport sys\nimport collections\nimport glob\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n#sagemaker dependencies\n\nimport boto3\nimport sagemaker\nfrom sagemaker import get_execution_role\nfrom sagemaker.image_uris import retrieve\nimport awswrangler as wr', '#instantiate a sagemaker session\nsmclient = boto3.Session().client("sagemaker")\nregion = boto3.Session().region_name\n\n#IAM role that is a

In [14]:
#creating the experiment
experiment_name = f"ll-failure-classification-{create_date}"
%store experiment_name

try:
    my_experiment = Experiment.load(experiment_name=experiment_name)
    print(f"Experiment loaded {experiment_name}: SUCESS")
except Exception as e:
    if "ResourceNotFound" in str(e):
        my_experiment = Experiment.create(
            experiment_name = experiment_name,
            description="Classification PrM Experiment",
            tags=[{"Key" : "my-experiments","Value":"exp"}],
            sagemaker_boto_client=smclient,
        )
        print(f"Experiment creation {experiment_name} : SUCESS")
    

Stored 'experiment_name' (str)
Experiment creation ll-failure-classification-22-02-01-15-40-34 : SUCESS


In [17]:
with Tracker.create(display_name="training",sagemaker_boto_client=smclient) as tracker:
    tracker.log_input(name="prm-dataset",media_type="s3/uri",value=path_to_train_data_prm)


In [15]:
print(path_to_train_data_prm)

s3://ml-predictivemaintainaince-28-01-2022/train/train.csv


In [ ]:
#Setting output path
lr_output_path = f"s3://{bucket}/{exp_prefix}/output/lr_default"
container = sagemaker.image_uris.retrieve(
    framework = "linear-learner",region=region,version="1",image_scope="training"
)

#creating the trail component and the first experiment for linear learner
training_trial_component = tracker.trial_component
trial_name_1 = trial_name = f"linear-learner-lr-training-job-{create_date}-1"

#create the trial if it doesn't exist

try:
    my_trial = Trial.load(trial_name=trial_name_1)
    print(f"Loaded existing trial : {trial_name}")
except Exception as e:
    if "ResourceNotFound" in str(e):
        my_trial = Trial.create(experiment_name=experiment_name,trial_name = trial_name_1)
        print(f"Create trial {my_trial.trial_name} : Sucessful")
        
        print(f"Creating logistic regression estimator \n")
        lr= sagemaker.estimator.Estimator(
            container,
            role,
            instance_count = 1,
            instance_type = "ml.c4.xlarge",
            output_path=lr_output_path,
            sagemaker_session=sess,
            enable_sagemaker_metrics=True,
        )
        
        lr.set_hyperparameters(
            predictor_type="binary_classifier",
            loss = "logistic",
            epochs=20,
            mini_batch_size=99
        )
        
        lr.fit(
            inputs=data_channel,
            experiment_config={
                "ExperimentName":my_experiment.experiment_name,
                "TrialName" : my_trial.trial_name,
                "TrialComponentDisplayName": "ll-lr-training-job"
            },
            logs=True
        )

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.


Create trial linear-learner-lr-training-job-22-02-01-15-40-34-1 : Sucessful
Creating logistic regression estimator 



INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: linear-learner-2022-02-02-15-29-12-886


2022-02-02 15:29:13 Starting - Starting the training job...
2022-02-02 15:29:36 Starting - Preparing the instances for trainingProfilerReport-1643815752: InProgress
......
2022-02-02 15:30:37 Downloading - Downloading input data..
2022-02-02 15:31:37 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[02/02/2022 15:31:34 INFO 140657601865536 integration.py:636] worker started
[02/02/2022 15:31:34 INFO 140657601865536] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init

In [25]:
data_channel

{'train': <sagemaker.inputs.TrainingInput at 0x7f23f418acd0>,
 'validation': <sagemaker.inputs.TrainingInput at 0x7f23ce59aa50>}

In [38]:
#Training the data with Hinge loss
svm_output_path = f"s3://{bucket}/{exp_prefix}/output/svm_default"
trial_name_2 = f"linear-lerner-svm-{create_date}-2-2"

#create the trial if it doesn't exist

try:
    my_trial = Trial.load(trial_name=trial_name_2)
    print(f"Loaded existing trial : {trial_name_2}")
except Exception as e:
    print("Inside exception")
    if "ResourceNotFound" in str(e):
        my_trial = Trial.create(experiment_name=experiment_name,trial_name=trial_name_2)
        print(f"Create trail {my_trial.trial_name} : SUCESSFUL")
        
        svm = sagemaker.estimator.Estimator(
            container,
            role,
            instance_count=1,
            instance_type="ml.c4.xlarge",
            output_path=svm_output_path,
            sagemaker_sesion=sess,
            enable_sagemaker_metrics=True,
        
        )
        
        svm.set_hyperparameters(
            predictor_type="binary_classifier",loss="hinge_loss",epochs=20,mini_batch_size=99
        )
        
        svm.fit(
            inputs=data_channel,
            experiment_config={
                "ExperimentName": my_experiment.experiment_name,
                "TrialName": my_trial.trial_name,
                "TrialComponentDisplayName": "ll-svm-training-job-hl",
            },
            logs=True
            
        )
    else:
        print("Resource found", e)

Inside exception
Create trail linear-lerner-svm-22-02-01-15-40-34-2-2 : SUCESSFUL


INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: linear-learner-2022-02-02-16-01-32-677


2022-02-02 16:01:32 Starting - Starting the training job...
2022-02-02 16:01:48 Starting - Preparing the instances for trainingProfilerReport-1643817692: InProgress
......
2022-02-02 16:03:02 Downloading - Downloading input data......
2022-02-02 16:04:00 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[02/02/2022 16:03:55 INFO 139830304876352 integration.py:636] worker started
[02/02/2022 16:03:55 INFO 139830304876352] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', '

In [44]:
trial_component_analytics = ExperimentAnalytics(experiment_name=my_experiment.experiment_name)
analytic_table = trial_component_analytics.dataframe()
analytic_table = analytic_table[
    [
        "TrialComponentName",
        "DisplayName",
        "validation:recall - Avg",
        "validation:binary_classification_accuracy - Avg",
        "validation:roc_auc_score - Avg",
        "train:objective_loss - Avg",
        "validation:objective_loss:final - Avg",
        "validation:objective_loss - Avg",
        "validation:binary_f_beta - Avg",
        "validation:precision - Avg",
        "Trials",
        "Experiments",
    ]
]

analytic_table.sort_values(
    ["validation:binary_classification_accuracy - Avg", "validation:binary_f_beta - Avg"],
    ascending=False,
)

AttributeError: 'NoneType' object has no attribute 'items'

                                  TrialComponentName             DisplayName  \
0  linear-learner-2022-02-02-16-01-32-677-aws-tra...  ll-svm-training-job-hl   
1  linear-learner-2022-02-02-15-29-12-886-aws-tra...      ll-lr-training-job   

   validation:recall - Avg  validation:binary_classification_accuracy - Avg  \
0                 0.489796                                         0.847778   
1                 0.581633                                         0.842222   

   validation:roc_auc_score - Avg  train:objective_loss - Avg  \
0                        0.726823                    0.215480   
1                        0.784895                    0.422724   

   validation:objective_loss:final - Avg  validation:objective_loss - Avg  \
0                               0.230429                         0.244350   
1                               0.454241                         0.463627   

   validation:binary_f_beta - Avg  validation:precision - Avg  \
0                        0.5

--------------------------------------------------------------------

Hyper parameter tuning

In [15]:
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter
from botocore.exceptions import ClientError

In [46]:
prm_tuning_job_name = f"11-svm-tuning-job"

In [ ]:
svm_output_path = f"s3://{bucket}/{exp_prefix}/output/tuning/svm_default"

# create the tuning job if it doesn't exist
try:
    svm_tune = sagemaker.estimator.Estimator(
        container,
        role,
        instance_count=1,
        instance_type="ml.c4.xlarge",
        output_path=svm_output_path,
        sagemaker_session=sess,
    )

    svm_tune.set_hyperparameters(
        predictor_type="binary_classifier", loss="hinge_loss", epochs=20, mini_batch_size=99
    )

    hyperparameter_ranges = {
        "learning_rate": ContinuousParameter(0.01, 0.5, scaling_type="Logarithmic")
    }

   
    my_tuner = HyperparameterTuner(
        estimator=svm_tune,  
        objective_metric_name="validation:binary_classification_accuracy",
        hyperparameter_ranges=hyperparameter_ranges,
        max_jobs=20,
        max_parallel_jobs=2,
        strategy="Random",
    )

    # start hyperparameter tuning job
    my_tuner.fit(inputs=data_channel, include_cls_metadata=False, job_name=prm_tuning_job_name)
    print(f"Create tuning job {prm_tuning_job_name}: SUCCESSFUL")
except ClientError as e:
    if "ResourceInUse" in str(e):
        my_tuner = HyperparameterTuner.attach(prm_tuning_job_name)
        print(f"Attach tuning job {prm_tuning_job_name}: SUCCESSFUL")

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating hyperparameter tuning job with name: 11-svm-tuning-job


...................................................................................................................................................................................................................................................................................................................................................................................................

In [17]:
boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=my_tuner.latest_tuning_job.job_name
)["HyperParameterTuningJobStatus"]

NameError: name 'my_tuner' is not defined

In [51]:
tune_analytics = sagemaker.HyperparameterTuningJobAnalytics(
    my_tuner.latest_tuning_job.job_name
).dataframe()

In [18]:
list_tuning_jobs_response = smclient.list_hyper_parameter_tuning_jobs(
    SortBy="CreationTime", SortOrder="Descending"
    
)

print(f'Found {len(list_tuning_jobs_response["HyperParameterTuningJobSummaries"])} tuning jobs')


Found 2 tuning jobs


In [19]:
tuning_jobs = list_tuning_jobs_response["HyperParameterTuningJobSummaries"]
most_recently_created_tuning_job = tuning_jobs[0]
tuning_job_name = most_recently_created_tuning_job["HyperParameterTuningJobName"]
experiment_name = my_experiment.experiment_name
tune_trial_name = tuning_job_name + "-trial"
%store tune_trial_name

print(f"Associate all training jobs created by {tuning_job_name} with trial  {tune_trial_name}")

NameError: name 'my_experiment' is not defined

In [30]:
try:
    tune_trial = Trial.load(trial_name=tune_trial_name)
    print(f"Loaded existing trial: {tune_trial_name}")
except Exception as e:
    if "ResourceNotFound" in str(e):
        tune_trial = Trial.create(experiment_name=experiment_name, trial_name=tune_trial_name)
        print(f"Create trial {tune_trial.trial_name} : SUCESS")

Loaded existing trial: 11-svm-tuning-job-trial


In [6]:
from sagemaker import HyperparameterTuningJobAnalytics, Session
from smexperiments.search_expression import Filter, Operator, SearchExpression
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent


In [57]:
%store tuning_job_name

Stored 'tuning_job_name' (str)


In [5]:
%store -r
%store

Stored variables and their in-db values:
create_date                         -> '22-02-01-15-40-34'
data_channel                        -> {'train': <sagemaker.inputs.TrainingInput object a
dw_output_path_prm                  -> 's3://ml-predictivemaintainaince-28-01-2022/export
exp_prefix                          -> 'sagemaker-experiments/linear-learner-22-02-01-15-
experiment_name                     -> 'll-failure-classification-22-02-01-15-40-34'
features_created_prm                -> True
path_to_test_data_prm               -> 's3://ml-predictivemaintainaince-28-01-2022/test/t
path_to_test_x_data_prm             -> 's3://ml-predictivemaintainaince-28-01-2022/test/t
path_to_train_data_prm              -> 's3://ml-predictivemaintainaince-28-01-2022/train/
path_to_valid_data_prm              -> 's3://ml-predictivemaintainaince-28-01-2022/valida
tune_trial_name                     -> '11-svm-tuning-job-trial'
tuning_job_name                     -> '11-svm-tuning-job'


In [10]:
tuning_analytics = HyperparameterTuningJobAnalytics(tuning_job_name,sess)

In [11]:
training_job_summaries = tuning_analytics.training_job_summaries()
training_job_arns = list(map(lambda x:x["TrainingJobArn"],training_job_summaries))
print(f"Found {len(training_job_arns)} training jobs for hyperparameter tuning job {tuning_job_name}")

Found 20 training jobs for hyperparameter tuning job 11-svm-tuning-job


In [21]:
import time
from datetime import datetime, timezone

creation_time = most_recently_created_tuning_job["CreationTime"]
print(creation_time)
creation_time = creation_time.astimezone(timezone.utc)
creation_time = creation_time.strftime("%Y-%m-%dT%H:%M:%SZ")
print(creation_time)

2022-02-02 16:34:40.951000+00:00
2022-02-02T16:34:40Z


In [23]:
created_after_filter = Filter(
    name="CreationTime",
    operator=Operator.GREATER_THAN_OR_EQUAL,
    value=str(creation_time)
)

In [24]:
print(created_after_filter)

Filter(name='CreationTime',operator='GreaterThanOrEqualTo',value='2022-02-02T16:34:40Z')


In [26]:
# the training job names contain the tuning job name (and the training job name is in the source arn)

source_arn_filter = Filter(
    name="Source.SourceArn",
    operator=Operator.CONTAINS,
    value=tuning_job_name
)

source_type_filter = Filter(
    name = "Source.SourceType",
    operator=Operator.EQUALS,
    value = "SageMakerTrainingJob"
)

In [27]:
search_expression = SearchExpression(
    filters=[created_after_filter,source_arn_filter,source_type_filter]
)

#Search iterates over every page of resilts by default
trial_component_search_results = list(
    TrialComponent.search(search_expression=search_expression,sagemaker_boto_client=smclient)
)

In [28]:
print(f"Found {len(trial_component_search_results)} trial components")

Found 20 trial components


In [31]:
for tc in trial_component_search_results:
    print(f"Associating trial component {tc.trial_component_name}  with trial {tune_trial.trial_name}")
    
    tune_trial.add_trial_component(tc.trial_component_name)
    
    time.sleep(0.5)

Associating trial component 11-svm-tuning-job-019-d27680bc-aws-training-job  with trial 11-svm-tuning-job-trial
Associating trial component 11-svm-tuning-job-020-f08bf10c-aws-training-job  with trial 11-svm-tuning-job-trial
Associating trial component 11-svm-tuning-job-018-d0fb9cdd-aws-training-job  with trial 11-svm-tuning-job-trial
Associating trial component 11-svm-tuning-job-017-a95ac93b-aws-training-job  with trial 11-svm-tuning-job-trial
Associating trial component 11-svm-tuning-job-015-43224350-aws-training-job  with trial 11-svm-tuning-job-trial
Associating trial component 11-svm-tuning-job-016-0c01dd0a-aws-training-job  with trial 11-svm-tuning-job-trial
Associating trial component 11-svm-tuning-job-014-a75722eb-aws-training-job  with trial 11-svm-tuning-job-trial
Associating trial component 11-svm-tuning-job-013-e9e3c5dd-aws-training-job  with trial 11-svm-tuning-job-trial
Associating trial component 11-svm-tuning-job-012-564d7c5b-aws-training-job  with trial 11-svm-tuning-jo

In [34]:
df_1 = tuning_analytics.dataframe()

In [36]:
df_1

AttributeError: 'NoneType' object has no attribute 'items'

    learning_rate                 TrainingJobName TrainingJobStatus  \
0        0.074463  11-svm-tuning-job-020-f08bf10c         Completed   
1        0.044962  11-svm-tuning-job-019-d27680bc         Completed   
2        0.104923  11-svm-tuning-job-018-d0fb9cdd         Completed   
3        0.017198  11-svm-tuning-job-017-a95ac93b         Completed   
4        0.336931  11-svm-tuning-job-016-0c01dd0a         Completed   
5        0.019972  11-svm-tuning-job-015-43224350         Completed   
6        0.098860  11-svm-tuning-job-014-a75722eb         Completed   
7        0.204969  11-svm-tuning-job-013-e9e3c5dd         Completed   
8        0.011807  11-svm-tuning-job-012-564d7c5b         Completed   
9        0.029147  11-svm-tuning-job-011-2cd1b49a         Completed   
10       0.075813  11-svm-tuning-job-010-793ff713         Completed   
11       0.141202  11-svm-tuning-job-009-302422f2         Completed   
12       0.033013  11-svm-tuning-job-008-c420f772         Completed   
13    

In [37]:
print("-"*100)

----------------------------------------------------------------------------------------------------


PREDICTION

In [45]:
my_tuner = boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName="11-svm-tuning-job"
)

In [47]:
print(my_tuner)

{'HyperParameterTuningJobName': '11-svm-tuning-job', 'HyperParameterTuningJobArn': 'arn:aws:sagemaker:ap-south-1:832173187970:hyper-parameter-tuning-job/11-svm-tuning-job', 'HyperParameterTuningJobConfig': {'Strategy': 'Random', 'HyperParameterTuningJobObjective': {'Type': 'Maximize', 'MetricName': 'validation:binary_classification_accuracy'}, 'ResourceLimits': {'MaxNumberOfTrainingJobs': 20, 'MaxParallelTrainingJobs': 2}, 'ParameterRanges': {'IntegerParameterRanges': [], 'ContinuousParameterRanges': [{'Name': 'learning_rate', 'MinValue': '0.01', 'MaxValue': '0.5', 'ScalingType': 'Logarithmic'}], 'CategoricalParameterRanges': []}, 'TrainingJobEarlyStoppingType': 'Off'}, 'TrainingJobDefinition': {'StaticHyperParameters': {'_tuning_objective_metric': 'validation:binary_classification_accuracy', 'epochs': '20', 'loss': 'hinge_loss', 'mini_batch_size': '99', 'predictor_type': 'binary_classifier'}, 'AlgorithmSpecification': {'TrainingImage': '991648021394.dkr.ecr.ap-south-1.amazonaws.com/li

In [48]:
batch_input = path_to_test_x_data_prm

In [49]:
batch_output = f"s3://{bucket}/transform/batch-inference"


In [50]:
from sagemaker.model import Model

In [53]:
import sagemaker
input_data_path = path_to_test_x_data_prm
output_data_path = batch_output
transform_job = sagemaker.transformer.Transformer(
    model_name = 'Predictive-Maintainaince',
    instance_count = 1,
    instance_type = 'ml.m4.xlarge',
    strategy = 'SingleRecord',
    output_path = output_data_path,
    base_transform_job_name='pred-maint-inference-pipelines-batch',
    sagemaker_session=sess)

In [54]:

transform_job.transform(data = input_data_path, 
                        content_type = 'text/csv')

INFO:sagemaker:Creating transform job with name: pred-maint-inference-pipelines-batch-2022-02-05-11-53-58-237


..................................Docker entrypoint called with argument(s): serve
Running default environment configuration script
[02/05/2022 11:59:31 INFO 140486062561088] loaded entry point class algorithm.serve.server_config:config_api
[02/05/2022 11:59:31 INFO 140486062561088] loading entry points
[02/05/2022 11:59:31 INFO 140486062561088] loaded request iterator application/json
[02/05/2022 11:59:31 INFO 140486062561088] loaded request iterator application/jsonlines
[02/05/2022 11:59:31 INFO 140486062561088] loaded request iterator application/x-recordio-protobuf
[02/05/2022 11:59:31 INFO 140486062561088] loaded request iterator text/csv
[02/05/2022 11:59:31 INFO 140486062561088] loaded response encoder application/json
[02/05/2022 11:59:31 INFO 140486062561088] loaded response encoder application/jsonlines
[02/05/2022 11:59:31 INFO 140486062561088] loaded response encoder application/x-recordio-protobuf
[02/05/2022 11:59:31 INFO 140486062561088] loaded response encoder text/csv

In [42]:
def evaluate_model(batch_file_path, test_labels, model_name, metrics=True):
    """
    Download batch predictions and iterate through results. Evaluate model results by comparing to actuals from test data.
    Return binary classification metrics.
    """

    # open batch prediction file and parse json to exctract predicted label
    with open(batch_file_path, "r") as f:
        for line in f:
            result = json.loads(line)
            print(result)
            predictions = pd.Series(
                [
                    result["predicted_label"]
                ]
            )

    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, predictions).sum()
    print(test_labels ," ",predictions) 
    print(1-test_labels, " ", predictions)
    fp = np.logical_and(1 - test_labels, predictions).sum()
    tn = np.logical_and(1 - test_labels, 1 - predictions).sum()
    fn = np.logical_and(test_labels, 1 - predictions).sum()

    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    f1 = 2 * precision * recall / (precision + recall)

    if metrics:
        print(pd.crosstab(test_labels, predictions, rownames=["actuals"], colnames=["predictions"]))
        print(f"Accuracy: {accuracy}")
        print(f"F1: {f1}")
        print(f"Recall: {recall}")
        print(f"Precision: {precision}")

    return {
        "TP": tp,
        "FP": fp,
        "FN": fn,
        "TN": tn,
        "Precision": precision,
        "Recall": recall,
        "Accuracy": accuracy,
        "F1": f1,
        "Model": model_name,
    }

In [57]:
!aws s3 cp s3://ml-predictivemaintainaince-28-01-2022/transform/batch-inference/test_x.csv.out .

download: s3://ml-predictivemaintainaince-28-01-2022/transform/batch-inference/test_x.csv.out to ./test_x.csv.out


In [43]:
# call evaluation function and inspect results
import json
test = pd.read_csv('test.csv', header=None)
test_y = test[0]
evaluate_model("test_x.csv.out", test_y, "PrM-Classification-SVM", metrics=True)

{'predicted_label': 0, 'score': -2.630024433135986}
{'predicted_label': 0, 'score': -2.636324167251587}
{'predicted_label': 0, 'score': -2.317629337310791}
{'predicted_label': 0, 'score': -2.569161176681518}
{'predicted_label': 0, 'score': -2.503008365631103}
{'predicted_label': 0, 'score': -2.52507734298706}
{'predicted_label': 0, 'score': -2.592943668365478}
{'predicted_label': 0, 'score': -2.394976139068603}
{'predicted_label': 0, 'score': -2.662158489227295}
{'predicted_label': 0, 'score': -2.692114353179931}
{'predicted_label': 0, 'score': -1.028131961822509}
{'predicted_label': 0, 'score': -0.97673487663269}
{'predicted_label': 0, 'score': -1.044942617416381}
{'predicted_label': 0, 'score': -1.134082078933715}
{'predicted_label': 0, 'score': -0.938485741615295}
{'predicted_label': 0, 'score': -0.986536741256713}
{'predicted_label': 0, 'score': -1.173613548278808}
{'predicted_label': 0, 'score': -1.120817422866821}
{'predicted_label': 0, 'score': -1.101728081703186}
{'predicted_la

ImportError: cannot import name 'FuncType' from 'pandas._typing' (/opt/conda/lib/python3.7/site-packages/pandas/_typing.py)

In [41]:
from pandas._typing import FuncType

ImportError: cannot import name 'FuncType' from 'pandas._typing' (/opt/conda/lib/python3.7/site-packages/pandas/_typing.py)

In [40]:
pip install pandas --upgrade

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
